In [1]:
import os
import re
import math
import pickle
import numpy as np
import pandas as pd
from statistics import mean
from sklearn.metrics import cohen_kappa_score

In [5]:
# with open("Summary_Ratings_A1.pkl", "rb") as f:
#     rater1_ratings = pickle.load(f)   
# with open("Summary_Ratings_A2.pkl", "rb") as f:
#     rater2_ratings = pickle.load(f)
# with open("midprocessing/filtered_rating_neg3.pkl", "rb") as f:
#     negatives = pickle.load(f)

with open("./midprocessing/Revised_Ratings.pkl", "rb") as f:
    ratings = pickle.load(f)
to_toss = {}


In [3]:
# def valence(num):
#     if num > 3:
#         return 1
#     elif num == 3:
#         return 0
#     elif num < 3:
#         return -1

In [4]:
test

{'testNegativeParseCases': [313],
 'setGenJarDir': [314],
 'test': [312],
 'updateGain': [311],
 'appendDeclarations': [311],
 'getUserNameFromCookie': [168],
 'add': [186],
 'updateSchema': [106],
 'exportXVRL': [133],
 'isInvalidEmailLink': [189],
 'testOneTwoThreeCreateCycle': [155],
 'addPKColumn': [310],
 'equals': [310],
 'getImageWithSource': [310],
 'swapItems': [143],
 'asMap': [143, 314, 136, 316],
 'testAddCountryWithSequenceGenerator': [168,
  133,
  129,
  315,
  314,
  117,
  191,
  136,
  139,
  317,
  147,
  182,
  176],
 'split': [133],
 'configBalanceRanking': [314, 106, 319, 176]}

In [10]:

def filter(person, func, ratings_list):
    global to_toss
    name = func.split('_')[0] # just the function name, without the ID number
    try:
        acc = int(ratings_list[0])
        mis = int(ratings_list[1])
        unn = int(ratings_list[2])
        rea = int(ratings_list[3])
    except:
        print(person, func)
    if mean([acc, rea]) <= 2 and mean([mis, unn]) >= 4:
        if name in to_toss:
            to_toss[name].append(int(person))
        else:
            to_toss[name] = [int(person)]
        print("delete data for: ", person, func)


In [33]:
### Manual override for these summaries. They should be included
ratings['147']['add_4114383'][3] = '3'
ratings['147']['getNAGString_43137003'][3] = '3'
ratings['147']['actionPerformed_12725774'][3] = '3'
ratings['147']['testSetWelcomeMsg_33719869'][3] = '3'
ratings['168']['appendDeclarations_36405409'][3] = '3'
ratings['133']['equals_10222579'][3] = '3'
ratings['311']['actionPerformed_12725774'][3]= '3'



In [17]:
# with open("./midprocessing/Revised_Ratings.pkl", "wb") as f:
#     pickle.dump(ratings, f)
# to_toss = {}
to_toss

{'testNegativeParseCases': [313, 147, 319, 313],
 'setGenJarDir': [314, 314],
 'test': [312, 312],
 'updateGain': [311, 191, 311],
 'appendDeclarations': [311, 168, 311],
 'getUserNameFromCookie': [168, 168],
 'add': [186, 186],
 'updateSchema': [106, 106],
 'exportXVRL': [133, 133],
 'isInvalidEmailLink': [189, 189],
 'testOneTwoThreeCreateCycle': [155, 155],
 'addPKColumn': [310, 310],
 'equals': [310, 310],
 'getImageWithSource': [310, 310],
 'swapItems': [143, 143],
 'asMap': [143, 314, 136, 316],
 'testAddCountryWithSequenceGenerator': [168,
  133,
  129,
  315,
  314,
  117,
  191,
  136,
  139,
  317,
  147,
  182,
  176],
 'split': [133],
 'configBalanceRanking': [314, 106, 319, 176]}

In [11]:
for person, functions in ratings.items():
    for func, rate_list in functions.items():
        filter(person, func, rate_list)


delete data for:  147 testNegativeParseCases_31696447
delete data for:  314 setGenJarDir_31789275
delete data for:  319 testNegativeParseCases_31696447
delete data for:  312 test_10_bug2689872_19498280
delete data for:  313 testNegativeParseCases_31696447
delete data for:  191 updateGain_26215341
delete data for:  168 appendDeclarations_36405409
delete data for:  168 getUserNameFromCookie_1694531
delete data for:  186 add_4114383
delete data for:  106 updateSchema_4627680
delete data for:  133 exportXVRL_45047585
delete data for:  189 isInvalidEmailLink_49250848
delete data for:  155 testOneTwoThreeCreateCycle_19507735
delete data for:  311 appendDeclarations_36405409
delete data for:  311 updateGain_26215341
delete data for:  310 addPKColumn_51577053
delete data for:  310 equals_10222579
delete data for:  310 getImageWithSource_1782360
delete data for:  143 swapItems_36634895


In [5]:
r1_positive_ratings = []
r1_negative_ratings = []
r2_positive_ratings = []
r2_negative_ratings = []

r1_accuracy = []
r1_missing = []
r1_unnecessary = []
r1_readable = []
r2_accuracy = []
r2_missing = []
r2_unnecessary = []
r2_readable = []


accuracy_differences = []
missing_differences = []
unnecessary_differences = []
readable_differences = []

to_toss = {}

for person, functions in rater1_ratings.items():
    for func, ratings in functions.items():
        name = func.split('_')[0]
        other_ratings = rater2_ratings[person][func]
        
        ratings = [int(r) for r in ratings if r != '']
        other_ratings = [int(r) for r in other_ratings if r != '']
        if len(ratings) != 4 or len(other_ratings) != 4:
            continue
        try:
            r1_acc, r1_mis, r1_unn, r1_rea = valence(ratings[0]), valence(ratings[1]), valence(ratings[2]), valence(ratings[3])
            r2_acc, r2_mis, r2_unn, r2_rea = valence(other_ratings[0]), valence(other_ratings[1]), valence(other_ratings[2]), valence(other_ratings[3])
            
            if r1_acc != 0 and r2_acc != 0:
                r1_accuracy.append(r1_acc)
                r2_accuracy.append(r2_acc)
            
            if r1_mis != 0 and r2_mis != 0:
                r1_missing.append(r1_mis)
                r2_missing.append(r2_mis)
            
            if r1_unn != 0 and r2_unn != 0:
                r1_unnecessary.append(r1_unn)
                r2_unnecessary.append(r2_unn)
            
            if r1_rea != 0 and r2_rea != 0:
                r1_readable.append(r1_rea)
                r2_readable.append(r2_rea)
            # acc_dif = abs(r1_acc - r2_acc)
            # mis_dif = abs(r1_mis - r2_mis)
            # unn_dif = abs(r1_unn - r2_unn)
            # rea_dif = abs(r1_rea - r2_rea)
            
            # accuracy_differences.append(acc_dif)
            # missing_differences.append(mis_dif)
            # unnecessary_differences.append(unn_dif)
            # readable_differences.append(rea_dif)
            
            # r1_pos = valence(mean([r1_acc, r1_rea]))
            # r1_neg = valence(mean([r1_mis, r1_unn]))
            # r2_pos = valence(mean([r2_acc, r2_rea]))
            # r2_neg = valence(mean([r2_mis, r2_unn]))
            
            # r1_positive_ratings.append(r1_pos)
            # r1_negative_ratings.append(r1_neg)
            # r2_positive_ratings.append(r2_pos)
            # r2_negative_ratings.append(r2_neg)
            
            
            
            if mean([r1_acc, r1_rea, r2_acc, r2_rea]) <= 2 and mean([r1_mis, r1_unn, r2_mis, r2_unn]) >= 4:
                if func in to_toss:
                    to_toss[name].append(int(person))
                else:
                    to_toss[name] = [int(person)]
                print("delete data for: ", person, func)
            # r1_positive_ratings.append(math.ceil(r1_pos))
            # r1_negative_ratings.append(math.ceil(r1_neg))
            # r2_positive_ratings.append(math.ceil(r2_pos))
            # r2_negative_ratings.append(math.ceil(r2_neg))
            # r1_positive_ratings.append(ratings[0])
            # r2_positive_ratings.append(other_ratings[0])
        except:
            print(func)
            print(ratings)
            print(other_ratings)
            
print(len(r1_negative_ratings), len(r2_negative_ratings))
print(len(r1_positive_ratings), len(r2_positive_ratings))


0 0
0 0


In [13]:
reading_toss = pd.read_csv("toss_these_summaries.csv")
superfixpath = "abstract_fixation_counts"
superdurpath = "abstract_fixation_durations"
# whichtask = os.listdir(superfixpath)
test = []
pathname = "reading_ratings"
datadir = os.listdir(pathname)
for file in datadir:
    name = re.sub(".csv", "", file)
    if name in list(reading_toss['function_name']):
        fidx = list(np.where(reading_toss['function_name'] == name))[0][0]
        author = reading_toss.loc[fidx, 'author']
        who_saw = pd.read_csv(f"{pathname}/{file}")
        delete_idx = list(np.where(who_saw['summary_author'] == author))[0]
        temp = list(who_saw.loc[delete_idx, 'pid'])
        for person in temp:
            if name in to_toss:
                to_toss[name].append(int(person))
            else:
                to_toss[name] = [int(person)]


In [21]:
# with open("midprocessing/to_toss.pkl", "wb") as f:
#     pickle.dump(to_toss, f)
# iterate through data to toss
# find 

In [16]:
print(writefiles)

['createCacheFile.csv', 'getBackCommand22.csv', 'getNAGString.csv', 'getBackCommand2.csv', 'getRemoteObject.csv', 'setPhoto.csv', 'testValidateSeparationCount.csv', 'saveToFile.csv', 'testSetWelcomeMsg.csv', 'testMoveRFWithNullContainer.csv', 'compareTo.csv', 'makeUniqueParagraphForGlobalWithLength.csv', 'populateNewList.csv', 'visitRetStmt.csv', 'saveSetting.csv', 'clearFieldersFromField.csv', 'goToRegistration.csv', 'getClassNameForLookAndFeel.csv', 'listen.csv', 'testSetExample.csv', 'testCookieGreaterThan.csv', 'showLatestPlan.csv', 'messageSent.csv', 'testGetEmail.csv', 'handleHalt.csv', 'go.csv', 'abstractMatrix3D.csv', 'deleteCascade.csv', 'selectBracketingEntries.csv', 'invalidateSession.csv', 'onAttach.csv', 'getEffect.csv', 'getTargetServiceName.csv', 'setSecurityMode.csv', 'play.csv', 'getMenuAdministracion.csv', 'searchRecipe.csv', 'removeService.csv', 'getJSplitPane.csv', 'addRotation.csv', 'setUpMrj.csv', 'testInvoke.csv', 'store.csv', 'readFromFile.csv', 'evaluate.csv', 

In [20]:
readfiles = os.listdir("abstract_fixation_counts/reading")
writefiles = os.listdir("abstract_fixation_counts/writing")
for k, v in to_toss.items():
    print(k)
    filename = k+".csv"
    if filename in readfiles:
        task = "reading"
    elif filename in writefiles:
        task = "writing"
    fdf = pd.read_csv(f"abstract_fixation_counts/{task}/{filename}")
    ddf = pd.read_csv(f"abstract_fixation_durations/{task}/{filename}")
    fmask = ~fdf['pid'].isin(v)
    dmask = ~ddf['pid'].isin(v)
    fdf = fdf[fmask]
    ddf = ddf[dmask]
    # os.system(f"mv abstract_fixation_counts/{task}/{filename} unfiltered_backup/fc_{filename}")
    # os.system(f"mv abstract_fixation_durations/{task}/{filename} unfiltered_backup/fd_{filename}")
    # fdf.to_csv(f"abstract_fixation_counts/{task}/{filename}", index=False)
    # ddf.to_csv(f"abstract_fixation_durations/{task}/{filename}", index=False)

testNegativeParseCases
setGenJarDir
test
updateGain
appendDeclarations
getUserNameFromCookie
add
updateSchema
exportXVRL
isInvalidEmailLink
testOneTwoThreeCreateCycle
addPKColumn
equals
getImageWithSource
swapItems
asMap
testAddCountryWithSequenceGenerator
split
configBalanceRanking


In [ ]:
readfiles = os.listdir("abstract_fixation_counts/reading")
writefiles = os.listdir("abstract_fixation_counts/writing")
for files in to_toss.items():
    print(k)
    filename = k+".csv"
    if filename in readfiles:
        task = "reading"
    elif filename in writefiles:
        task = "writing"
    df = pd.read_csv(f"abstract_fixation_counts/{task}/{filename}")
    # ddf = pd.read_csv(f"abstract_fixation_durations/{task}/{filename}")


In [4]:
which_task = os.listdir("abstract_fixation_counts")
for task in which_task:
    files = os.listdir(f"abstract_fixation_counts/{task}")
    data_points = 0
    for file in files:
        df = pd.read_csv(f"abstract_fixation_counts/{task}/{file}")
        data_points += len(df)
    print(f"{data_points} data points in the {task} condition")

661 data points in the writing condition
996 data points in the reading condition
